In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score
import re
import statistics 
import matplotlib.pyplot as plt
from scipy import stats

print("hey")

hey


In [4]:
# removeMostUsed tells if words used more than the provided percentage should be removed.
# if false, words used less then the percentage should be removed
def removeWords(dataset, removeMostUsed=True, percentage=0.5):
    numberEntries = len(dataset)
    neededAmount = numberEntries * percentage
    
    
    vocab = {}
    for text in dataset:
        rejex = re.compile('<.*?>')
        text = rejex.sub('', text)
        thisVocab = {}
        for word in re.split('\W+',text):
            # text.split(" ").split("-"):
            regex = re.compile("[-'[:alpha:]0-9]")
            newWord = regex.sub('', word.lower())
            if(len(newWord)):
                if (newWord not in thisVocab):
                    thisVocab[newWord]=1
                    if (newWord not in vocab):
                        vocab[newWord]=1
                    else:
                        vocab[newWord]+=1
    # accedp only words used the needed amount or less
    if (removeMostUsed == True):
        newVocab = dict((k, v) for k, v in vocab.items() if v < neededAmount)
    # accedp only words used the needed amount or more
    else:
        newVocab = dict((k, v) for k, v in vocab.items() if v > neededAmount)
    newVocab = sorted(newVocab)
    ret = {}
    for word in vocab:
        if word in newVocab:
            ret[word] = newVocab.index(word)


    return ret  



In [5]:
def runMultipleTimes(dataset):
    f = open("bow_tfidf_1000_3.txt", "w")
    percentageAndMostused = [[True, 1.0], [True, .9], [True, .8], [True, .7], [False, .3], [False, .2], [False, .1]]
    # percentageAndMostused2 = [[ True, 1.0, True], [ True, 1.0, False], [ True, .9, True], [ True, .9, False], [ True, .8, True], [ True, .8, False]]

    avgScores = []
    allScores = []
    sdScores = []
    avgF1Scores = []
    sdF1Scoress = []
    allF1Scores = []

    
    allLoss = []

    for elem in percentageAndMostused:
        # count = None
        # if (elem[2]):
        vocab = removeWords(dataset.review, removeMostUsed=elem[0], percentage=elem[1])
        count = CountVectorizer(vocabulary=vocab )
        # else:
        # if (elem[0] == True):
        #     count = CountVectorizer(max_df=elem[1])
        # else:
        #     count = CountVectorizer(min_df=elem[1])

        X = count.fit_transform(dataset['review'])
        tfidf = TfidfTransformer()
        X = tfidf.fit_transform(X)
        
        avgScore = 0
        avgF1Score = 0
        count = 30
        scores = []
        f1scores = []
        loss = []
        print(elem)

        for x in range(count):
            X_train, X_test, y_train, y_test = train_test_split(X, dataset['sentiment'], test_size=0.33)
            model = MLPClassifier(max_iter=300)
            model.fit(X_train, y_train)
            scores.append(model.score(X_test, y_test))
            y_pred = model.predict(X_test)
            f1scores.append(f1_score(y_test, y_pred, average='weighted'))
            loss.append(model.loss_)
            # print(scores[-1])
            
        avgScore = statistics.mean(scores)
        allScores.append(scores)
        avgF1Score = statistics.mean(f1scores)
        allF1Scores.append(f1scores)

        sdScore = statistics.stdev(scores)
        sdF1Score = statistics.stdev(scores)

        avgScores.append(avgScore)
        avgF1Scores.append(avgF1Score)
        sdScores.append(sdScore)
        sdF1Scoress.append(sdF1Score)

        allLoss.append(loss)
        avgLoss = statistics.mean(loss)
        sdLoss = statistics.stdev(loss)

        # if (elem[2]):
        #     f.write("mine\n")
        # else:
        #     f.write("sklearn\n")
        f.write("most used: " + str(elem[0]) + '\n')
        f.write("rate " + str(elem[1]) + '\n\n')

        f.write("Average Score " + str(avgScore) + '\n')
        f.write("Score sd " + str(sdScore) + '\n')
        f.write("All Scores" + str(scores) + '\n\n')

        f.write("Average Loss " + str(avgLoss) + '\n')
        f.write("Loss sd " + str(sdLoss) + '\n')
        f.write("All losses" + str(loss) + '\n\n')

        f.write("Average F1Score " + str(avgF1Score) + '\n')
        f.write("F1Score sd " + str(sdF1Score) + '\n')
        f.write("All F1 Scores" + str(f1scores) + '\n\n\n')
    
    figScore, axScore = plt.subplots()
    axScore.set_title('Scores')
    axScore.boxplot(allScores)
    legend = ["All Words", "rm 90% most used", "rm 80% most used", "rm 70% most used", "rm 30% least usesd", "rm 20% least usesd", "rm 10% least usesd"]

    xTicks = []
    for i in range(1, len(avgScores)+1):
        xTicks.append(i)
    x = xTicks
    plt.xticks(x, legend, rotation=45)
    plt.title('Scores')
    plt.show()
    
    figF1Score, axF1Score = plt.subplots()
    axF1Score.set_title('F1 Scores')
    axF1Score.boxplot(allF1Scores)
    
    plt.xticks(x, legend, rotation=45)
    plt.title('F1 Scores')
    plt.show()
    
    figLoss, axLoss = plt.subplots()
    axLoss.set_title('Losses')
    axLoss.boxplot(allLoss)
    plt.xticks(x, legend, rotation=45)
    plt.title('Loss')
    plt.show()

    for i in range(1, len(allScores)):
        # alternative{‘two-sided’, ‘less’, ‘greater’}, optional
        print(legend[0], legend[i], statistics.mean(allScores[0]), statistics.mean(allScores[1]))
        print(stats.ttest_ind(allScores[0], allScores[i], equal_var=False))



In [14]:
dataset = pd.read_csv('datasets/imdb.csv')
dataset.head()
dataset = dataset[:1000]
# runMultipleTimes(dataset)
vocab = removeWords(dataset.review, removeMostUsed=True, percentage=.005)


In [5]:
count = CountVectorizer(max_df=1.0)
count2 = CountVectorizer(vocabulary=(vocab))
X = count.fit_transform(dataset['review'])
XX = count2.fit_transform(dataset['review'])
x1 =  count.vocabulary_
x2 = count2.vocabulary_
# yes = []
# no = []
# for word in vocab:
#     if word not in x1:
#         no.append(word)
#     else:
#         yes.append(word)

NameError: name 'vocab' is not defined

In [13]:
X.shape

(1000, 17906)

In [7]:
tfidf = TfidfTransformer()

In [8]:
X = tfidf.fit_transform(X)

In [9]:
X.shape

(1000, 17625)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, dataset['sentiment'], test_size=0.33)

In [11]:
X_train.shape

(670, 17625)

In [12]:
X_test.shape

(330, 17625)

In [13]:
model = MLPClassifier(max_iter=300)
model.fit(X_train, y_train)

MLPClassifier(max_iter=300)

In [14]:
model.score(X_test, y_test)

0.8181818181818182

In [15]:
y_pred = model.predict(X_test)
f1_score(y_test, y_pred, average='weighted')

0.8181283835007063